# 1. Data Preparation
Before starting to train the model, some steps need to be done.

# 1.1. ROI Extraction
The images are large in size (above 2000 x 2000 pixels), so it is better to extract the area that is important to us by using a cropping function. This cropping process should be applied to both the image and the respective masks.

<div style="text-align: center;">
    <img src="Images/image-11.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>


* TASK:

In [1]:
import cv2
import os
from pathlib import Path
from PIL import Image

# Input and output directories
raw_imgs = Path("../images/raw_images").glob("*.jpg")
raw_labels = Path("../images/raw_labels").glob("*.png")
cropped_img_dir = Path("../images/") / "cropped_images"
cropped_label_dir = Path("../images/") / "cropped_labels"
cropped_img_dir.mkdir(exist_ok=True)
cropped_label_dir.mkdir(exist_ok=True)

def crop_imgs(input_imgs, output_dir) -> None:
   for img_path in input_imgs:
      #crop the center section ~> 1400W * 1840H
      with Image.open(img_path) as img:
         width, height = img.size
         left = (width - 1400) // 2
         top = (height - 1840) // 2
         right = left + 1400
         bottom = top + 1840
         cropped = img.crop((left, top, right, bottom))
         cropped.save(output_dir / img_path.name)
   return
   
crop_imgs(raw_imgs, cropped_img_dir)
crop_imgs(raw_labels, cropped_label_dir)

# 1.2. Image Format Converting 
It is recommended to convert the format of both the image and mask to TIFF format, which is suitable for the recommended Convolutional Neural Network (CNN) model.

* TASK:

In [2]:
### Code
##ConTif
cropped_imgs = Path("../images/cropped_images").glob("*.jpg")
cropped_labels = Path("../images/cropped_labels").glob("*.png")
tif_img_dir = Path("../images/") / "tif_images"
tif_label_dir = Path("../images/") / "tif_labels"
tif_img_dir.mkdir(exist_ok=True)
tif_label_dir.mkdir(exist_ok=True)

def convert_imgs_to_tif(input_imgs, output_dir) -> None:
    for img_path in input_imgs:
        # Convert it to Tif
        with Image.open(img_path) as img:
            tif_name = img_path.stem + ".tif"
            img.save(output_dir / tif_name, format="TIFF")
    return

convert_imgs_to_tif(cropped_imgs, tif_img_dir)
convert_imgs_to_tif(cropped_labels, tif_label_dir)

# 1.3. Image Augmentation

After extracting the ROI and converting the image and mask to TIFF format, we need to increase our dataset size using augmentation techniques. Here's how you can implement this method, ensuring that the image name and the respective mask name are the same:

* Define Augmentation Parameters: Determine the augmentation techniques to apply, such as rotation, flipping, scaling, etc.

* Loop Through Images: Iterate through each image and its corresponding mask.

* Apply Augmentation: Apply the defined augmentation techniques to both the image and its mask.

* Save Augmented Images: Save the augmented images and their masks with the same names as the original images and masks.

<div style="text-align: center;">
    <img src="Images/image-12.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK:

In [6]:
import numpy as np
import random
from scipy.ndimage import rotate, gaussian_filter
from skimage.util import random_noise
from skimage.exposure import adjust_gamma, rescale_intensity

# Define functions for each operation

def rotation(image, seed):
    random.seed(seed)
    angle = random.uniform(-180, 180)
    return rotate(image, angle, reshape=False, mode='reflect')

def h_flip(image, seed):
    random.seed(seed)
    return np.fliplr(image) if random.random() > 0.5 else image

def v_flip(image, seed):
    random.seed(seed)
    return np.flipud(image) if random.random() > 0.5 else image

def add_noise(image, seed):
    np.random.seed(seed)
    noisy_img = random_noise(image, mode='gaussian', var=0.01)
    return np.clip(noisy_img, 0, 1)

def adjust_brightness(image, seed):
    random.seed(seed)
    gamma = random.uniform(0.8, 1.2)
    bright_img = adjust_gamma(image, gamma)
    return np.clip(bright_img, 0, 1)

def adjust_contrast(image, seed):
    random.seed(seed)
    factor = random.uniform(0.8, 1.2)
    contrast_img = rescale_intensity(image, in_range='image', out_range=(0, 1))
    mean = np.mean(contrast_img)
    contrast_img = np.clip((contrast_img - mean) * factor + mean, 0, 1)
    return contrast_img

def gaussian_blur(image, seed):
    random.seed(seed)
    sigma = random.uniform(0.5, 1.5)
    blurred_img = gaussian_filter(image, sigma=(sigma, sigma, 0))  # No blur on channels
    return blurred_img


* TASK:

In [ ]:
from tqdm import tqdm

###Use the functions to implement the augmentation for both images and masks
transformations = {
    'rot': rotation,
    'hflip': h_flip,
    'vflip': v_flip,
    "noise": add_noise,
    "brightness": adjust_brightness,
    "contrast": adjust_contrast,
    # "blur": gaussian_blur
}
tif_imgs = Path("../images/tif_images").glob("*.tif")
tif_labels = Path("../images/tif_labels").glob("*.tif")
augmented_img_dir = Path("../images/") / "augmented_images"
augmented_label_dir = Path("../images/") / "augmented_labels"
augmented_img_dir.mkdir(exist_ok=True)
augmented_label_dir.mkdir(exist_ok=True)

for img_path, label_path in zip(tif_imgs, tif_labels):
    print(f"{img_path.name}")
    img = np.array(Image.open(img_path))
    label = np.array(Image.open(label_path))

    for t_name, t_func in tqdm(transformations.items()):
        seed = random.randint(0, 10000)  # Same seed for image and mask

        aug_img = t_func(img, seed)
        aug_msk = t_func(label, seed)

        # Save augmented versions
        aug_img_pil = Image.fromarray(aug_img.astype(np.uint8))
        aug_label_pil = Image.fromarray(aug_msk.astype(np.uint8))
        aug_img_pil.save(augmented_img_dir / f"{t_name}_{img_path.name}")
        aug_label_pil.save(augmented_label_dir / f"{t_name}_{label_path.name}")

image0003446.tif


  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:02<00:00,  2.74it/s]


image0002687.tif


100%|██████████| 6/6 [00:02<00:00,  2.92it/s]


...

100%|██████████| 6/6 [00:03<00:00,  1.84it/s]


image0003362.tif


100%|██████████| 6/6 [00:02<00:00,  2.68it/s]


image0002743.tif


 17%|█▋        | 1/6 [00:01<00:09,  1.97s/it]